# Chapter 3: Customized Tools in Strands Agents

## Introduction to Custom Tools

While the built-in tools provide a solid foundation for many tasks, the real power of Strands Agents comes from the ability to create custom tools tailored to your specific needs. In this chapter, we'll explore how to create, refine, and extend your own tools to enhance your agents' capabilities.

Custom tools allow you to:
- Connect agents to your own APIs and services
- Create domain-specific functionality
- Build tools for specialized data processing
- Interface with databases and external systems
- Implement business logic specific to your use case

Throughout this chapter, we'll use the Nova Lite model (`us.amazon.nova-lite-v1:0`) as specified for our course.

## Setup and Prerequisites

Let's start by installing the necessary packages:

In [ ]:
# Install strands-agents and strands-agents-tools if you haven't already
!pip install -U strands-agents strands-agents-tools

## Creating Your First Custom Tool

The simplest way to create a custom tool in Strands Agents is to use the `@tool` decorator with a Python function:

In [ ]:
from strands import Agent, tool

# Define a simple custom tool
@tool
def word_counter(text: str) -> dict:
    """
    Count the occurrence of each word in a text.
    
    Args:
        text (str): The input text to analyze
        
    Returns:
        dict: A dictionary mapping words to their occurrence count
    """
    # Remove punctuation and convert to lowercase
    import string
    text = text.lower()
    for punct in string.punctuation:
        text = text.replace(punct, ' ')
    
    # Split into words and count occurrences
    words = text.split()
    word_counts = {}
    for word in words:
        word_counts[word] = word_counts.get(word, 0) + 1
    
    return word_counts

# Create an agent with our custom tool
agent = Agent(
    model="us.amazon.nova-lite-v1:0",
    tools=[word_counter],
    system_prompt="You are an assistant that can analyze text and provide word statistics."
)

Let's test our custom tool by asking the agent to analyze some text:

In [ ]:
response = agent("""
Please analyze the following paragraph and tell me which words appear most frequently:

"Strands Agents is a powerful framework for building AI agents using Python. 
With Strands Agents, you can create intelligent agents that leverage language models 
like Nova Pro to perform complex tasks. The framework provides tools for enhancing 
your agents with various capabilities, making it easy to build sophisticated AI 
applications."
""")

## Anatomy of a Custom Tool

Let's break down the key components of a custom tool in Strands Agents:

1. **The `@tool` Decorator**: This tells Strands that the function should be treated as a tool
2. **Type Annotations**: Used to define the expected input and output types
3. **Docstring**: Crucial for the agent to understand when and how to use the tool
4. **Implementation**: The actual code that executes when the tool is used

### The Importance of Good Docstrings

The docstring is particularly important because it helps the AI model understand:
- What the tool does
- When to use it
- What inputs it expects
- What outputs it produces

In [ ]:
@tool
def text_sentiment_analyzer(text: str) -> dict:
    """
    Analyze the sentiment of a text using a simple lexicon-based approach.
    
    This tool classifies text sentiment by counting positive and negative words from a 
    predefined lexicon. It's useful for getting a quick sentiment assessment of user feedback,
    product reviews, or social media comments.
    
    Args:
        text (str): The text to analyze for sentiment. Should be in English and ideally 
                    contain at least 10 words for better accuracy.
        
    Returns:
        dict: A dictionary containing:
            - 'sentiment': The overall sentiment ('positive', 'negative', or 'neutral')
            - 'score': A sentiment score from -1.0 (very negative) to 1.0 (very positive)
            - 'positive_words': List of positive words found in the text
            - 'negative_words': List of negative words found in the text
    
    Note: This is a simplified sentiment analyzer for demonstration purposes.
    """
    # Simple lexicons
    positive_words = {
        'good', 'great', 'excellent', 'wonderful', 'fantastic', 'amazing', 'love', 'best',
        'happy', 'joy', 'positive', 'helpful', 'beautiful', 'perfect', 'recommend'
    }
    
    negative_words = {
        'bad', 'terrible', 'awful', 'horrible', 'poor', 'worst', 'hate', 'dislike', 
        'negative', 'disappointed', 'disappointing', 'useless', 'waste', 'broken'
    }
    
    # Normalize text
    text = text.lower()
    words = ''.join(c if c.isalnum() else ' ' for c in text).split()
    
    # Find positive and negative words
    found_positive_words = [word for word in words if word in positive_words]
    found_negative_words = [word for word in words if word in negative_words]
    
    # Calculate sentiment score
    pos_count = len(found_positive_words)
    neg_count = len(found_negative_words)
    total_count = pos_count + neg_count
    
    if total_count == 0:
        sentiment = "neutral"
        score = 0.0
    else:
        score = (pos_count - neg_count) / total_count
        if score > 0.1:
            sentiment = "positive"
        elif score < -0.1:
            sentiment = "negative"
        else:
            sentiment = "neutral"
    
    # Return results
    return {
        'sentiment': sentiment,
        'score': round(score, 2),
        'positive_words': found_positive_words,
        'negative_words': found_negative_words
    }

Now, let's create an agent with both of our custom tools and test how it uses them:

In [ ]:
text_analysis_agent = Agent(
    model="us.amazon.nova-lite-v1:0",
    tools=[word_counter, text_sentiment_analyzer],
    system_prompt="You are an assistant that specializes in text analysis."
)

response = text_analysis_agent("""
Please analyze these two product reviews and tell me about their sentiment and language usage:

Review 1: "I absolutely love this product! It's easy to use and works perfectly. 
The design is beautiful and it's the best purchase I've made this year."

Review 2: "This was a terrible disappointment. The product arrived broken and 
customer service was unhelpful. Save your money and avoid this awful product."
""")

## Tool Input and Output Types

Strands supports a variety of input and output types for tools. Let's explore some common patterns:

In [ ]:
@tool
def text_formatter(text: str, format_type: str, max_length: int = 100) -> str:
    """
    Format text according to specified formatting options.
    
    Args:
        text (str): The input text to format
        format_type (str): The type of formatting to apply. Valid options are:
                          'uppercase', 'lowercase', 'title_case', 'sentence_case', 'truncate'
        max_length (int, optional): Maximum length when using 'truncate' format. Default is 100.
        
    Returns:
        str: The formatted text
    """
    format_type = format_type.lower()
    
    if format_type == 'uppercase':
        return text.upper()
    elif format_type == 'lowercase':
        return text.lower()
    elif format_type == 'title_case':
        return text.title()
    elif format_type == 'sentence_case':
        return '. '.join(s.capitalize() for s in text.split('. '))
    elif format_type == 'truncate':
        if len(text) <= max_length:
            return text
        return text[:max_length] + '...'
    else:
        return f"Error: Unknown format type '{format_type}'."

### Working with Complex Data Types

In [ ]:
from typing import List, Dict, Any

@tool
def data_summarizer(data: List[Dict[str, Any]], fields_to_summarize: List[str]) -> Dict[str, Any]:
    """
    Summarize numerical fields in a list of dictionaries (e.g., records or JSON data).
    
    Args:
        data (List[Dict]): A list of dictionaries (records) to summarize
        fields_to_summarize (List[str]): List of field names to include in the summary
        
    Returns:
        Dict: A dictionary containing summary statistics for each specified field:
              - count: Number of records
              - min: Minimum value
              - max: Maximum value
              - avg: Average value
              - sum: Sum of values
    """
    result = {}
    
    # Check if there's any data to summarize
    if not data:
        return {"error": "No data provided"}
    
    # Process each requested field
    for field in fields_to_summarize:
        # Check if the field exists in the records
        valid_values = []
        for record in data:
            if field in record and isinstance(record[field], (int, float)):
                valid_values.append(record[field])
        
        # Generate statistics if we have valid values
        if valid_values:
            result[field] = {
                "count": len(valid_values),
                "min": min(valid_values),
                "max": max(valid_values),
                "avg": sum(valid_values) / len(valid_values),
                "sum": sum(valid_values)
            }
        else:
            result[field] = {"error": "No valid values found"}
    
    return result

Let's create an agent with our new tools and test them:

In [ ]:
advanced_tools_agent = Agent(
    model="us.amazon.nova-lite-v1:0",
    tools=[text_formatter, data_summarizer],
    system_prompt="You are an assistant with advanced text and data processing capabilities."
)

response = advanced_tools_agent("""
I have a couple tasks for you:

1. Format the following text in title case:
"strands agents is a powerful framework for building AI assistants with advanced capabilities."

2. Summarize this sales data:
[
    {"product": "Laptop", "price": 1200, "units_sold": 45, "rating": 4.7},
    {"product": "Phone", "price": 800, "units_sold": 125, "rating": 4.5},
    {"product": "Tablet", "price": 350, "units_sold": 85, "rating": 4.2},
    {"product": "Headphones", "price": 200, "units_sold": 155, "rating": 4.8},
    {"product": "Monitor", "price": 400, "units_sold": 60, "rating": 4.6}
]

For the sales data, I want statistics on price, units_sold, and rating fields.
""")

## Adding Error Handling to Custom Tools

Robust error handling is crucial for production-grade tools. Here's an example with proper error handling:

In [ ]:
@tool
def robust_text_analyzer(text: str, analysis_type: str) -> Dict[str, Any]:
    """
    Analyze text using different analysis methods with robust error handling.
    
    Args:
        text (str): The text to analyze
        analysis_type (str): The type of analysis to perform. Valid options are:
                            'word_count', 'char_count', 'sentence_count', 'readability'
        
    Returns:
        dict: Results of the analysis, or an error message if something went wrong
    """
    try:
        # Input validation
        if not isinstance(text, str):
            return {"error": "Input text must be a string"}
        
        if not text.strip():
            return {"error": "Input text is empty"}
        
        analysis_type = analysis_type.lower()  # Normalize input
        valid_types = {'word_count', 'char_count', 'sentence_count', 'readability'}
        
        if analysis_type not in valid_types:
            return {
                "error": f"Invalid analysis type: '{analysis_type}'.", 
                "valid_options": list(valid_types)
            }
        
        # Perform the requested analysis
        if analysis_type == 'word_count':
            words = text.split()
            return {
                "total_words": len(words),
                "unique_words": len(set(words))
            }
            
        elif analysis_type == 'char_count':
            return {
                "total_chars": len(text),
                "letters": sum(c.isalpha() for c in text),
                "digits": sum(c.isdigit() for c in text),
                "spaces": sum(c.isspace() for c in text)
            }
            
        elif analysis_type == 'sentence_count':
            import re
            sentences = re.split(r'[.!?](?:\s|$)', text)
            sentences = [s for s in sentences if s.strip()]
            return {
                "sentence_count": len(sentences),
                "avg_sentence_length": len(text) / max(len(sentences), 1)
            }
            
        elif analysis_type == 'readability':
            words = text.split()
            if not words:
                return {"error": "Cannot calculate readability for empty text"}
                
            avg_word_length = sum(len(word) for word in words) / len(words)
            
            if avg_word_length < 4:
                difficulty = "Easy"
            elif avg_word_length < 6:
                difficulty = "Medium"
            else:
                difficulty = "Complex"
                
            return {
                "avg_word_length": avg_word_length,
                "difficulty_estimate": difficulty
            }
    
    except Exception as e:
        # Catch and report any unexpected errors
        return {"error": f"Analysis failed with error: {str(e)}"}

# Create an agent with our robust tool
robust_agent = Agent(
    model="us.amazon.nova-lite-v1:0",
    tools=[robust_text_analyzer],
    system_prompt="You are an assistant that can analyze text with high reliability."
)

## Tools with External Dependencies

Custom tools can leverage external libraries to provide advanced functionality. Here's a simplified example that would use NLTK for text processing:

In [ ]:
# This is an example - you'd need to install NLTK first with:
# !pip install nltk

@tool
def nlp_tool(text: str, operation: str) -> Dict[str, Any]:
    """
    Process text using NLP techniques.
    
    Args:
        text: The text to process
        operation: The NLP operation to perform ('tokenize', 'pos_tag', 'entities')
        
    Returns:
        Dict containing the results of the NLP processing
    """
    try:
        import nltk
        # You would need to download nltk data packages first:
        # nltk.download('punkt')
        # nltk.download('averaged_perceptron_tagger')
        
        if operation == 'tokenize':
            return {
                "tokens": nltk.word_tokenize(text),
                "sentences": nltk.sent_tokenize(text)
            }
        elif operation == 'pos_tag':
            tokens = nltk.word_tokenize(text)
            return {"tagged": nltk.pos_tag(tokens)}
        else:
            return {"error": f"Unknown operation: {operation}"}
    except Exception as e:
        return {"error": str(e)}

# Note: This is just an example and would require NLTK to be installed

## Best Practices for Custom Tools

When creating custom tools for Strands Agents, keep these best practices in mind:

1. **Write Clear Docstrings**: The docstring is how the model understands your tool's purpose and parameters. Be detailed and explicit.
2. **Use Type Annotations**: Type hints help the model understand what data types your tool expects and returns.
3. **Handle Errors Gracefully**: Return informative error messages instead of allowing exceptions to propagate.
4. **Keep Tools Focused**: Each tool should do one thing well rather than trying to handle multiple unrelated tasks.
5. **Provide Input Validation**: Validate inputs early to prevent processing invalid data.
6. **Use Descriptive Names**: Choose function and parameter names that clearly convey their purpose.
7. **Return Structured Data**: When possible, return structured data (dictionaries, lists) that's easy for the model to work with.
8. **Consider Performance**: Optimize tools that might be called frequently or process large amounts of data.
9. **Test Thoroughly**: Test your tools with various inputs, including edge cases, to ensure they behave as expected.
10. **Version Dependencies**: If your tool depends on external libraries, specify version requirements.

## Summary

In this chapter, we've explored:

- Creating custom tools using the `@tool` decorator
- The importance of clear docstrings and type annotations
- Working with various input and output types
- Adding robust error handling to tools
- Integrating external libraries into custom tools
- Best practices for tool development

Custom tools are one of the most powerful features of Strands Agents, allowing you to extend your agents' capabilities in virtually any direction. By creating well-designed tools, you can enable your agents to perform complex, domain-specific tasks that go far beyond simple text generation.

In the next chapter, we'll explore how to integrate Strands Agents with the Model Context Protocol (MCP), which enables even more powerful tool integration and interoperability.

## Exercises

1. Create a custom tool that can load and parse CSV files, then perform basic data analysis operations on the data.
2. Build a tool that connects to a weather API and returns weather forecasts for a given location.
3. Develop a tool that performs image operations (e.g., resizing, format conversion) using a library like Pillow.
4. Create a custom tool that interacts with a database to store and retrieve information.
5. Design a tool that validates and formats structured data like JSON or XML.